# IMPORT

In [ ]:
import torch
from torch.nn import functional as F
import torch.nn as nn
import json
import sentencepiece as spm
import pandas as pd

# HYPERPARAMETERS

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
n_layer = 8
n_head = 8
n_embd = 512
dropout = 0.2
block_size = 128

batch_size = 32
learning_rate = 1e-4
max_iters = 5000
eval_iters = 50
eval_interval = 100

spm_vocab = 10000

# GET DATA

## COMBINE THE PRETRAINING FILE

In [ ]:
file_path = "../data/raw/pretraining/train-00001-of-00002.parquet"
file = pd.read_parquet(file_path)

file.shape

In [ ]:
with open("../data/raw/pretraining/pre.txt", 'a', encoding='utf-8') as f:
    for i in range(file.shape[0]):
        f.write(file.iloc[i].text + " ")

## GET THE FINAL PRETRAINING FILE

In [ ]:
with open("../data/raw/pretraining/pre.txt", 'r', encoding='utf-8') as f:
        text = f.read()
file = None

In [ ]:
len(text.split())

## GET THE SUMMARY DATA

In [ ]:
train = pd.read_parquet("../data/raw/summary/train-00000-of-00002.parquet", columns=["article", "summary"])

In [ ]:
indices = []
for i in range(train.shape[0]):
    if len(train.iloc[i]['article'].split()) <= 128:
        indices.append(i)

In [ ]:
train = train.iloc[indices]

In [ ]:
train = train.reset_index(drop=True)

## CREATE THE SPM TOKENIZER

In [ ]:
spm.SentencePieceTrainer.train(input="../data/raw/pretraining/pre.txt", model_prefix='spm_model', vocab_size=spm_vocab)

In [ ]:
sp = spm.SentencePieceProcessor(model_file='../data/tokenizer/spm_model.model')

In [ ]:
sp.get_piece_size()

In [ ]:
sp.encode("")

In [ ]:
sp.decode([4])

In [ ]:
[9, 8] + [7] + [6, 5]

## TOKENIZE THE PRETRAINING TEXT ON THE NEW TOKENIZER

In [ ]:
text = sp.encode(text, out_type=int)

In [ ]:
type(text)

In [ ]:
# TRAIN TEST SPLIT
text = torch.tensor(text, dtype=torch.long)

# SPLIT SIZE
n = int(0.9*len(text)) # first 90% will be train, rest val

# SPLITS
train_data = text[:n]
val_data = text[n:]

## TOKENIZE THE SUMMARY TEXT ON THE NEW TOKENIZER

In [ ]:
X = []
y = []

for i in range(train.shape[0]):
    enc = sp.encode(train.iloc[i]['article'], out_type=int)
    enc = enc[:block_size]
    if len(enc) < block_size:
        enc += [1] * (block_size - len(enc))
    X.append(enc)
    
    enc = sp.encode(train.iloc[i]['summary'], out_type=int)
    enc = enc[:block_size]
    if len(enc) < block_size:
        enc += [1] * (block_size - len(enc))
    y.append(enc)

# full_text = None
# summary = None

In [ ]:
# TRAIN TEST SPLIT

# SPLIT SIZE
n = int(0.9*len(X)) # first 90% will be train, rest val

# SPLITS
train_full = torch.asarray(X[:n])
train_summary = torch.asarray(y[:n])
test_full = torch.asarray(X[n:])
test_summary = torch.asarray(y[n:])

X = None
y = None

# MODEL

In [ ]:
# data loading for pretraining step
def get_batch(step, split):
    if step == "pretraining":
        data = train_data if split == 'train' else val_data
        
        ix = torch.randint(len(data) - block_size, (batch_size,))
        
        x = torch.stack([data[i:i+block_size] for i in ix])
        y = torch.stack([data[i+1:i+block_size+1] for i in ix])
        
        x, y = x.to(device), y.to(device)

    elif step == "summary":
        
        
        if split == "train":
            ix = torch.randint(0, len(train_full), (batch_size,))
    
            x = train_full[ix]
            y = train_summary[ix]
            
            x, y = x.to(device), y.to(device)

        else:

            ix = torch.randint(0, len(test_full), (batch_size,))
        
            x = test_full[ix]
            y = test_summary[ix]
            
            x, y = x.to(device), y.to(device)

    return x, y

In [ ]:
from rouge_score import rouge_scorer

# Initialize the scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

def evaluate_rouge(prediction, summaries):
    
    rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

    for ref, gen in zip(summaries, prediction):
        scores = scorer.score(ref, gen)
        rouge_scores['rouge1'].append(scores['rouge1'].fmeasure)
        rouge_scores['rouge2'].append(scores['rouge2'].fmeasure)
        rouge_scores['rougeL'].append(scores['rougeL'].fmeasure)

    avg_rouge_scores = {key: sum(values)/len(values) for key, values in rouge_scores.items()}

    print(f"ROGUE: {avg_rouge_scores}")

def clean_and_decode(tokens, sp):
    # Remove -1 tokens and decode
    tokens = [token for token in tokens if token != -1]
    return sp.decode(tokens)


In [ ]:
def estimate_loss(step):
    out = {}
    
    model.eval()
    for split in ['train', 'val']:
        
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            
            X, Y = get_batch(step, split)

            logits = model(X)

            # predictions = logits.argmax(dim=-1)
            # # Decode predictions and targets to text
            # decoded_predictions = [clean_and_decode(pred.tolist(), sp) for pred in predictions]
            # decoded_targets = [clean_and_decode(target.tolist(), sp) for target in Y]
            # evaluate_rouge(decoded_predictions, decoded_targets)

            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = Y.view(B*T)

            valid_mask = targets != -1
            targets = targets[valid_mask]
            logits = logits[valid_mask]

            loss = F.cross_entropy(logits, targets)
            losses[k] = loss.item()

        out[split] = losses.mean()
    
    model.train()
    
    return out

In [ ]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape

        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        v = self.value(x) # (B,T,C)

        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5                       # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1)                                 # (B, T, T)
        wei = self.dropout(wei)

        # perform the weighted aggregation of the values
        out = wei @ v                                                # (B, T, T) @ (B, T, C) -> (B, T, C)
        
        return out

In [ ]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [ ]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.GELU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [ ]:
# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(sp.get_piece_size(), n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, sp.get_piece_size())

    def forward(self, idx):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        return logits

In [ ]:
def generate(idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            model.eval()
            logits = model(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

# DEFINE

In [ ]:
model = BigramLanguageModel().to(device)

# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)

# TRAINING

In [ ]:
for iter in range(max_iters*4):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss("pretraining")
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch("pretraining", 'train')

    # get prediction
    logits = model(xb)

    B, T, C = logits.shape
    logits = logits.view(B*T, C)
    targets = yb.view(B*T)
    loss = F.cross_entropy(logits, targets)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

In [ ]:
context = "Today"
context = sp.encode(context, out_type=int)
context = torch.tensor(context, dtype=torch.long, device=device).unsqueeze(0)
a = generate(context, max_new_tokens=10*3)[0].tolist()
sp.decode(a)

In [ ]:
torch.save(model, 'model.pth')

# SUMMARY TRAINING

In [ ]:
model = torch.load('model.pth')

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)

In [ ]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss("summary")
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch("summary", 'train')

    # get prediction
    logits = model(xb)

    B, T, C = logits.shape
    logits = logits.view(B*T, C)
    targets = yb.view(B*T)

    valid_mask = targets != -1
    targets = targets[valid_mask]
    logits = logits[valid_mask]


    loss = F.cross_entropy(logits, targets)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# TESTING PIPELINE MODEL

In [5]:
import torch
import tiktoken

In [15]:
X = torch.load("../data/processed/summary/val_tensor.pt")

In [16]:
# Load the GPT-2 tokenizer
tokenizer = tiktoken.get_encoding('gpt2')

In [17]:
print(tokenizer.decode(X[3][0].tolist()))

Summarize the following article.

### Article:
inter milan are already relishing the prospect of revenge over european champions juventus when they meet in the italian cup quarter - finals next week .

### Summary:
inter ready for revenge on juventus<|endoftext|>~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~